In [ ]:
import pandas as pd
import numpy as np
import networkx as nx

In [ ]:
head_lst = ["conn_id", "user_id", "node_id", "timestamp_in", "timestamp_out"]
data = pd.read_csv("../dataset\stable\contact_network\mtl wifi\isf_wifidog_anonymised_data_nogeo_2004-08-27_2010-03-07.csv", names=head_lst)
data = data.dropna()
data.head()

In [ ]:
data['timestamp_in'] = pd.to_datetime(data['timestamp_in'])
data['timestamp_out'] = pd.to_datetime(data['timestamp_out'])
data.info()

In [ ]:
df_date = data[data['timestamp_in'] < '2006-11-30'] # uncomment for wifi 1
# df_date = data[(data['timestamp_in'] > '2007-07-01') & (data['timestamp_in'] < '2008-02-26')] # uncomment for wifi 2
# df_date = data[data['timestamp_in'] > '2010-03-05'] # uncomment for wifi 3

In [ ]:
user_id_map_to_num = {k:v for v,k in enumerate(df_date.user_id.unique())}

In [ ]:
df_date['user_id'] = df_date['user_id'].apply(lambda x: user_id_map_to_num[x])

In [ ]:
groups = df_date.groupby("node_id").filter(lambda x: len(x) > 1)

In [ ]:
groups = groups.groupby("node_id")

In [ ]:
# %%time
edges = []
neighbours = []
i = 0
for name, group in groups:
    print(i)
    for index, row in group.iterrows():
        time_in = row['timestamp_in']
        time_out = row['timestamp_out']
        df_rows_to_check = group[(group.index > index) & (group['user_id'] != row['user_id'])]
        neighbours = df_rows_to_check[((df_rows_to_check['timestamp_out']>=time_in) & (df_rows_to_check['timestamp_out']<=time_out)) | 
                        ((df_rows_to_check['timestamp_in']>=time_in) & (df_rows_to_check['timestamp_in']<=time_out))]['user_id'].values
        if len(neighbours) > 0:
            user = row['user_id']
            edges.append([[user, neighbour] for neighbour in neighbours.tolist()])
    i += 1

In [ ]:
edges = [ e for edge_lst in edges for e in edge_lst]

In [ ]:
user_id_map_to_num = {k:v for v,k in enumerate(np.unique(np.array(edges)))}

In [ ]:
edges_labelled = [[user_id_map_to_num[n] for n in e] for e in edges ]

In [ ]:
edges_unique = np.unique(np.array(edges_labelled), axis=0)

In [ ]:
import csv

with open('../dataset\stable\contact_network/mtl wifi/mtl_wifi.edgelist', 'w', newline="") as f:
    writer = csv.writer(f, delimiter=' ')
    writer.writerows(edges_unique)

In [ ]:
df_edges = pd.read_csv('../dataset\stable\contact_network/mtl wifi/mtl_wifi.edgelist', sep=" ", names=["i", "j"])

In [ ]:
G = nx.read_edgelist("../dataset\stable\contact_network/mtl wifi/mtl_wifi.edgelist", nodetype=int)

In [ ]:
largest_cc = max(nx.connected_components(G), key=len)
print("Mtl wifi number of connected componenets: ", nx.number_connected_components(G), len(largest_cc))

In [ ]:
C = G.subgraph(largest_cc).copy()
C = nx.convert_node_labels_to_integers(C)

In [ ]:
nx.write_edgelist(C, "../dataset\stable\contact_network/mtl wifi/mtl_wifi.edgelist")